# Script for setting up Google Earth Engine Authentication and loading the NDVI per tree
In order for this to run you must have done the steps from the notebooks load_tree_geojson and merge_datasets already to have the necessary data sets in the data folder.

In [31]:
import ee
ee.Initialize(project='ee-ninaimmenroth')

# Load NDVI per tree
Create a table per tree with monthly aggregate ndvi (mean of the best mosaic values in the buffer zone).
The time is limited to the year 2022 for now as it had most watering events.
I used only a sample of the trees in Charlottenburg-Wilmersdorf and Pankow since these are the districts with most waterings in 2022.

In [32]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

In [33]:
# Load GeoJSON file of merged data set
# df_trees = gpd.read_file('data/berlin_strassenbaeume.geojson')
df_trees = gpd.read_file(
    "data/berlin_trees_with_watering_2022.gpkg",
    layer="trees_watering"
)

print(df_trees.head())

                               id_x              gisid              pitid  \
0  strassenbaeume.00008100_000bbafb  00008100_000bbafb  00008100:000bbafb   
1  strassenbaeume.00008100_000bbafd  00008100_000bbafd  00008100:000bbafd   
2  strassenbaeume.00008100_000bbafe  00008100_000bbafe  00008100:000bbafe   
3  strassenbaeume.00008100_000bbaff  00008100_000bbaff  00008100:000bbaff   
4  strassenbaeume.00008100_000bbb00  00008100_000bbb00  00008100:000bbb00   

  standortnr kennzeich              namenr                art_dtsch  \
0         93     01414  Fritz-Reuter-Allee      Pyramiden-Hainbuche   
1         91     01414  Fritz-Reuter-Allee  Berg-Ahorn, Weiss-Ahorn   
2         90     01414  Fritz-Reuter-Allee  Berg-Ahorn, Weiss-Ahorn   
3         89     01414  Fritz-Reuter-Allee  Berg-Ahorn, Weiss-Ahorn   
4         88     01414  Fritz-Reuter-Allee  Berg-Ahorn, Weiss-Ahorn   

                         art_bot gattung_deutsch   gattung  ... stammumfg  \
0  Carpinus betulus 'Fastigiata' 

In [34]:
df_ch_wilm = df_trees[
    df_trees['bezirk'].isin(['Charlottenburg-Wilmersdorf', 'Pankow']) &
    df_trees['has_watering']
]
print(df_ch_wilm.head())
df_ch_wilm.shape

                                   id_x              gisid              pitid  \
12785  strassenbaeume.00008100_000c09d7  00008100_000c09d7  00008100:000c09d7   
12792  strassenbaeume.00008100_000c09e9  00008100_000c09e9  00008100:000c09e9   
12793  strassenbaeume.00008100_000c09ea  00008100_000c09ea  00008100:000c09ea   
12829  strassenbaeume.00008100_000c0a20  00008100_000c0a20  00008100:000c0a20   
12830  strassenbaeume.00008100_000c0a21  00008100_000c0a21  00008100:000c0a21   

      standortnr kennzeich            namenr        art_dtsch  \
12785         48     01380   Friedbergstraße   Echter Rotdorn   
12792         30     01380   Friedbergstraße   Echter Rotdorn   
12793         29     01380   Friedbergstraße   Echter Rotdorn   
12829         49     01362  Fredericiastraße  Fächerblattbaum   
12830         48     01362  Fredericiastraße  Fächerblattbaum   

                                    art_bot gattung_deutsch    gattung  ...  \
12785  Crataegus laevigata 'Paul´s Scarlet'

(974, 27)

## split data because of throttling
we need to use smaller samples of data, so run everything first for the first part, then for the second and merge the sets later on.

In [48]:
df_ch_wilm_1 = df_ch_wilm.iloc[:500]
df_ch_wilm_2 = df_ch_wilm.iloc[500:]
print(len(df_ch_wilm_1), len(df_ch_wilm_2))

# comment this is to do everything for the second split! and rename the file so it won't overwrite
df_ch_wilm_1 = df_ch_wilm_2

500 474


In [49]:
df_ch_wilm.geometry.iloc[0].__geo_interface__

{'type': 'Point', 'coordinates': (13.291954046804742, 52.50364823686027)}

In [50]:
features = [
    ee.Feature(
        ee.Geometry(g.__geo_interface__),
        {"tree_id": pid}
    )
    for g, pid in zip(
        df_ch_wilm_1.geometry,
        df_ch_wilm_1.pitid
    )
]

trees_fc = ee.FeatureCollection(features)

In [51]:
trees_fc_buffered = trees_fc.map(lambda f: f.buffer(10))

In [52]:
def add_ndvi(img):
    ndvi = img.normalizedDifference(["B8", "B4"]).rename("NDVI")
    return img.addBands(ndvi)

s2 = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(trees_fc_buffered)
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
    .map(add_ndvi)
)

In [53]:
def monthly_ndvi_stats(year, month):
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, "month")

    monthly = s2.filterDate(start, end).select("NDVI")

    reducer = (
        ee.Reducer.mean()
        .combine(ee.Reducer.median(), True)
        .combine(ee.Reducer.stdDev(), True)
        .combine(ee.Reducer.count(), True)
    )

    stats_img = monthly.reduce(reducer)

    return stats_img.sampleRegions(
        collection=trees_fc_buffered,
        scale=10,
        geometries=False
    ).map(lambda f: f.set({"year": year, "month": month}))

In [54]:
def monthly_ndvi_quality(year, month):
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, "month")

    monthly = s2.filterDate(start, end)

    best_img = (
        monthly
        .qualityMosaic("NDVI")
        .select("NDVI")
        .rename("NDVI")
    )

    return best_img.reduceRegions(
        collection=trees_fc_buffered,
        reducer=ee.Reducer.mean().setOutputs(["NDVI"]),
        scale=10
    ).map(lambda f: f.set({
        "year": year,
        "month": month
    }))


In [55]:
months = list(range(3, 10))  # March–September
year = 2022

fc_list = [
    monthly_ndvi_quality(year, m)
    for m in months
]

ndvi_fc = ee.FeatureCollection(fc_list).flatten()


In [56]:
ndvi_fc_nogeom = ndvi_fc.map(
    lambda f: ee.Feature(
        None,
        f.toDictionary([
            "tree_id",
            "year",
            "month",
            "NDVI"
        ])
    )
)


In [57]:
ndvi_fc_nogeom.limit(1).getInfo()


{'type': 'FeatureCollection',
 'columns': {},
 'features': [{'type': 'Feature',
   'geometry': None,
   'id': '0_0',
   'properties': {'NDVI': 0.38483616264196274,
    'month': 3,
    'tree_id': '00008100:0017bed1',
    'year': 2022}}]}

In [58]:
import pandas as pd

# Convert Earth Engine FeatureCollection → client-side dict
data = ndvi_fc_nogeom.getInfo()

# Convert to pandas DataFrame
df = pd.DataFrame([f["properties"] for f in data["features"]])
df.head()
df.shape

(3318, 4)

In [59]:
df.to_csv("data/ndvi_CWP_2022_watering_p2.csv", index=False)